In [2]:
import cvxpy as cp
import numpy as np
import csv
from datetime import datetime

In [3]:
def DCOPF(Y,PGl,PGu,PD,thetaL,thetaU,CQ,CL,PF,slack,gens_to_buses):
    num_buses = len(Y)
    num_generators = PGl.shape[1]
    B = np.imag(Y)
    t = len(PD)

    PG = cp.Variable((t,num_generators)); # optimization variable: PG_i
    theta = cp.Variable((t,num_buses)); # optimization variable: theta_i

    objective = cp.Minimize(cp.sum(PG**2 * CQ.T + PG * CL.T)) # objective function

    constraints = [
        # Power generation constraints
        PG - PGu <= 0,
        -PG + PGl <= 0,

        #Bus phase angle constraints
        theta.T - thetaU <= 0,
        -theta.T + thetaL <= 0,

        #Power balance constraint
        theta * B + PD == PG * gens_to_buses,

        #Slack angle constraint   
        theta[:,slack] == 0
    ]
    
    # Line capacity constraints
    count = 0
    for i in range(num_buses):          
        for j in range(i+1, num_buses):
            constraints.append(B[i,j] * (theta[:,i] - theta[:,j]) - PF[:,count] <= 0)
            constraints.append(B[i,j] * (theta[:,j] - theta[:,i]) - PF[:,count] <= 0)
            count = count + 1

    # SOLVE IT
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    pf_opt = np.zeros((t,num_buses,num_buses))
    for i in range(num_buses):
        for j in range(i+1, num_buses):
            pf_opt[:,i,j] = B[i,j] * (theta[:,i].value - theta[:,j].value)
    
    cost = objective.value
    p_opt = PG.value
    theta_opt = theta.value
    lmp_opt = constraints[4].dual_value.reshape((t,num_buses))
    
    return pf_opt, cost, p_opt, theta_opt, lmp_opt

In [4]:
def load_load_profiles(filename, loads, start_date, num_hours):
    with open(filename) as load_profiles_file:
        reader = csv.reader(load_profiles_file)
        next(reader)
        col_to_bus = [7,4,2,3,0,5,6,1] # VERY FILE SPECIFIC, MAKE SURE IT'S CORRECT BEFORE LOADING FILE
        t = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            if t >= num_hours:
                break

            day = datetime.strptime(next_line[0], '%m/%d/%y')
            if day >= start_date:
                for col in range(2,len(next_line)):
                    loads[t,col_to_bus[col-2]] = float(next_line[col])
                t += 1
            

In [5]:
def load_cost_curves(filename, cost_curves_dict):
    with open(filename) as curves_file:
        reader = csv.reader(curves_file)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            q = float(next_line[1])
            l = float(next_line[2])
            c = float(next_line[3])
            cost_curves_dict[fuel] = (q,l,c)

In [6]:
def load_generators(filename, generator_dict, cost_curves_dict):
    with open(filename) as generators_file:
        reader = csv.reader(generators_file)
        next(reader)
        count = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            weather_zone = next_line[1]
            jan_mar = float(next_line[2])
            apr_sep = float(next_line[3])
            oct_dec = float(next_line[4])
            gen_id = "{} {}".format(weather_zone, fuel)

            generator_dict[gen_id] = {
                            'idx'   : count,
                            'bus'   : weather_zone,
                            'p_nom' : apr_sep,
                            'cq'    : cost_curves_dict[fuel][0] * SBASE * SBASE,
                            'cl'    : SBASE * cost_curves_dict[fuel][1],
                            'c0'    : 1 * cost_curves_dict[fuel][2],
            }
            count += 1

            if 'wind' in fuel:
                generator_dict[gen_id]['p_max_pu'] = wind_curve
            else:
                generator_dict[gen_id]['p_max_pu'] = 1

In [56]:
SBASE = 1e3 #MW
NUM_HOURS = 24
START_DATE = datetime.strptime('05/01/17', '%m/%d/%y')
NUM_BUSES = 9
wind_curve = np.random.rand((NUM_HOURS))
solar_curve = np.random.rand((NUM_HOURS))

# Useful data structures to transfer between bus indexes and names
regidxs = {"North" : 0,
         "West" : 1,
         "FarWest" : 2,
         "NorthCentral" : 3,
         "East" : 4,
         "SouthCentral" : 5,
         "South" : 6,
         "Coast" : 7,
         "Northwest" : 8}

regnames = ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']

# distances of transmission lines, in km
distances = {}
distances[('FarWest', 'South')] = 579
distances[('FarWest', 'West')] = 224
distances[('West', 'North')] = 195
distances[('North', 'NorthCentral')] = 198
distances[('East', 'NorthCentral')] = 146
distances[('East', 'Coast')] = 290
distances[('West', 'SouthCentral')] = 340
distances[('SouthCentral', 'Coast')] = 243
distances[('NorthCentral', 'SouthCentral')] = 241 #note: needs fixing
distances[('South', 'SouthCentral')] = 193
distances[('South', 'Coast')] = 391
distances[('Northwest', 'North')] = 200 # made up

# Construct Y-bus
impedance_per_km = .001j

Y = np.zeros((NUM_BUSES, NUM_BUSES),dtype=complex)
for b1 in range(NUM_BUSES):
    for b2 in range(b1+1, NUM_BUSES):
        if (regnames[b1],regnames[b2]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b1],regnames[b2])]*impedance_per_km)
        elif (regnames[b2],regnames[b1]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b2],regnames[b1])]*impedance_per_km)
        else:
            Y[b1,b2] = 0
        Y[b2,b1] = Y[b1,b2]
    Y[b1,b1] = -1*np.sum(Y[b1,:])
    
# set voltage angle constraints
thetal = -np.ones((NUM_BUSES,NUM_HOURS))
thetau = np.ones((NUM_BUSES,NUM_HOURS))
    
# Get cost curves
cost_curves_dict = {}
load_cost_curves('cost_quadratic_estimates.csv', cost_curves_dict)
    
# Get all the generators
generators_dict = {}
load_generators('zonal_generator_capacities_wo_biomass.csv', generators_dict, cost_curves_dict)
num_gens = len(generators_dict.keys())

# Gens to buses matrix
gens_to_buses = np.zeros((num_gens, NUM_BUSES))
PGl = np.zeros((NUM_HOURS,num_gens))
PGu = np.zeros((NUM_HOURS,num_gens))
CQ = np.zeros((1,num_gens))
CL = np.zeros((1,num_gens))

for gen in generators_dict.keys():
    gen_idx = generators_dict[gen]['idx']
    bus_idx = regidxs[generators_dict[gen]['bus']]
    gens_to_buses[gen_idx, bus_idx] = 1
    PGu[:,gen_idx] = generators_dict[gen]['p_nom']
    if 'WIND' in gen:
        PGu[:,gen_idx] = PGu[:,gen_idx] * wind_curve
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    elif 'SOLAR' in gen:
        PGu[:,gen_idx] = PGu[:,gen_idx] * solar_curve
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    CQ[0,gen_idx] = generators_dict[gen]['cq']
    CL[0,gen_idx] = generators_dict[gen]['cl']

PGl = PGl / SBASE
PGu = PGu / SBASE

line_capacity = 3000
PF = line_capacity * np.ones((NUM_HOURS, num_gens)) / SBASE
slack= 0

PD = np.zeros((NUM_HOURS,NUM_BUSES))
load_load_profiles('load_profiles_processed.csv', PD, START_DATE, NUM_HOURS)
PD = PD/SBASE
print('Total demand over time period: ', np.sum(PD)*SBASE)

Total demand over time period:  526094.8040000001


In [63]:
pf_opt, cost, p_opt, theta_opt, lmp_opt = DCOPF(Y, PGl, PGu, PD, thetal, thetau, CQ, CL, PF, slack, gens_to_buses)
#print(np.matmul(p_opt**2, CQ.T) + np.matmul(p_opt, CL.T))
#print('OPF:', pf_opt)
print('Total cost (optimizer):', cost)
print('Total cost (real): ', np.sum(np.matmul(p_opt, gens_to_buses) * lmp_opt))
print('Generation:', p_opt * SBASE)
#print('Angles:', theta_opt)
print('LMPs: ', lmp_opt)

Total cost (optimizer): -2551115.7048118943
Total cost (real):  11679298.698865524
Generation: [[ 7.76122656e+02  3.96159069e+02  2.58100009e+03  3.35069238e-05
   7.76122657e+02  3.96159070e+02  2.34986396e-06  3.96159070e+02
   1.25825055e+02  9.50135877e+02  6.50000001e+02  3.96159071e+02
  -3.61385334e-04  6.24593904e+01  9.14217682e+02  7.76122655e+02
   3.96159069e+02 -3.61385403e-04  2.40000009e+03  1.19542661e+00
   4.77002676e+02  3.96159068e+02  1.60910464e-05  9.24087475e+02
   7.76122655e+02  3.96159070e+02 -3.61385342e-04  4.28074299e+02
   6.70529128e+02  7.76122656e+02  3.96159070e+02 -3.61385582e-04
   7.48914923e+01  3.96159069e+02 -3.61385572e-04  7.92546819e+01
   1.13817185e+03]
 [ 4.69853626e+02  3.56116411e+02  2.58100009e+03  3.25796350e-05
   4.69853626e+02  3.56116411e+02  1.42265718e-06  3.56116410e+02
   1.18578967e+02  1.10670828e+03  4.69853626e+02  3.56116411e+02
  -3.63240831e-04  3.39763555e+01  1.10670822e+03  4.69853626e+02
   3.56116411e+02 -3.6324086

In [45]:
print('Angles:', theta_opt)

Angles: [[-3.22451171e-13 -8.26055764e-01 -7.60593913e-01 ...  1.29643095e-01
  -6.74164678e-01  9.99999999e-01]
 [-2.16031172e-14 -8.23650808e-01 -7.59745668e-01 ...  1.34554207e-01
  -6.63445060e-01  9.99999999e-01]
 [-2.22459149e-13 -8.24957311e-01 -7.66937657e-01 ...  1.33124151e-01
  -6.64843951e-01  9.99999999e-01]
 ...
 [ 2.02749893e-13 -7.64145338e-01 -6.10395645e-01 ...  6.18296460e-01
  -4.44710824e-01  1.00000000e+00]
 [ 1.70955485e-13 -7.71394620e-01 -6.36129681e-01 ...  5.41644697e-01
  -4.76462418e-01  1.00000000e+00]
 [ 1.56026837e-13 -7.77654301e-01 -6.51487502e-01 ...  4.72641125e-01
  -4.98092730e-01  1.00000000e+00]]


In [36]:
print(Y)

[[0.+1.51787102j 0.-0.51282051j 0.+0.j         0.-0.50505051j
  0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.-0.5j       ]
 [0.-0.51282051j 0.+1.25336673j 0.-0.44642857j 0.+0.j
  0.+0.j         0.-0.29411765j 0.+0.j         0.+0.j
  0.+0.j        ]
 [0.+0.j         0.-0.44642857j 0.+0.61914014j 0.+0.j
  0.+0.j         0.+0.j         0.-0.17271157j 0.+0.j
  0.+0.j        ]
 [0.-0.50505051j 0.+0.j         0.+0.j         0.+1.60491977j
  0.-0.68493151j 0.-0.41493776j 0.+0.j         0.+0.j
  0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         0.-0.68493151j
  0.+1.02975909j 0.+0.j         0.+0.j         0.-0.34482759j
  0.+0.j        ]
 [0.+0.j         0.-0.29411765j 0.+0.j         0.-0.41493776j
  0.+0.j         0.+1.63871276j 0.-0.51813472j 0.-0.41152263j
  0.+0.j        ]
 [0.+0.j         0.+0.j         0.-0.17271157j 0.+0.j
  0.+0.j         0.-0.51813472j 0.+0.94660076j 0.-0.25575448j
  0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.j         0.+0.j
  0.-0.3